# Reventar a las casas de apuestas

### Imports and options

In [50]:
import sqlite3
import time
from bs4 import BeautifulSoup
from abc import ABC
import Levenshtein as lev
import string

In [68]:
# Source: https://github.com/JeffSackmann/tennis_atp y https://github.com/JeffSackmann/tennis_wta
"""
atp, wta, all = set(), set(), set()
for row in open("atp_players.csv"):
    player = row.split(',')
#    if player[4] == "" or int(player[4][:4]) <= 1980:
#        continue
    atp.add((player[1] + ' ' + player[2]).upper())
for row in open("wta_players.csv"):
    player = row.split(',')
#    if player[4] == "" or int(player[4][:4]) <= 1980:
#        continue
    wta.add((player[1] + ' ' + player[2]).upper())
all_players = atp.union(wta)
print(f"ATP total: {len(atp)}")
print(f"WTA total: {len(wta)}")
print(f"ALL total: {len(all_players)}")
"""

ATP total: 54625
WTA total: 39179
ALL total: 93751


In [191]:
conn = sqlite3.connect("bets_database.db")
cursor = conn.cursor()

#If the table does not exists, we create it
if cursor.execute("PRAGMA table_info(TENNIS_PLAYERS)").fetchall() == []:
    tennis_players_table_query = """CREATE TABLE TENNIS_PLAYERS (
        name TEXT     UNIQUE,
        sex  TEXT (1) 
    );"""
    cursor.execute(tennis_players_table_query)
    conn.commit()

In [197]:
all_players = set([x[0].upper() for x in cursor.execute("SELECT name FROM TENNIS_PLAYERS").fetchall()])

In [207]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC # Wait for condition

options = webdriver.ChromeOptions()
options.add_argument('ignore-certificate-errors')
options.add_argument("--window-size=1920,1080")
options.add_argument('--headless')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

#options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(options=options)

In [176]:
class Bet():
    def __init__(self, teams, bets):
        self.teams = [x.strip().upper() for x in teams]
        self.real_teams = None
        self.bets = [float(x.replace(',', '.')) for x in bets]
        self.date_time = None
        self.link = None
        self.house = None
    
    def is_error(self):
        if len(self.teams) < 2 or len(self.bets) < 2:
            return True
        return False

    def __repr__(self):
        if self.is_error():
            return "ERROR BET"
        return f"{self.teams[0]} ({self.bets[0]}) x {self.teams[1]} ({self.bets[1]})"

In [5]:
class House(ABC):
    def link_sport(self, sport):
        pass
    def sport_bets(self, sport):
        pass

In [6]:
class Retabet(House):
    def __init__(self):
        self.link = "https://apuestas.retabet.es"
        self.sports = {"tennis" : "tenis-m8"}

    def link_sport(self, sport):
        return f"{self.link}/deportes/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        table = soup.find('ul', attrs={'class':'module__events-cont jbgroup'}) #TODO findAll
        ret_bets = []
        for row in table:
            row = row.find('a', attrs={'class':'jlink'})
            teams = row.find('div', attrs={'class':'module__event-info'})
            teams = teams.find('ul', attrs={'class':'module__event-players'})
            teams = list(map(lambda elem: elem.text, teams.findAll('li')))

            bets = row.find('div', attrs={'class':'list-events__element__bets'})
            bets = row.find('div', attrs={'class':'module__bets-list jbet'})
            bets = row.findAll('li', attrs={'class':'jo module__bets-li'})
            bets_list = []
            for bet, _ in zip(bets, range(2)):
                bet = bet.find('span', attrs={'class':'jpr module__bets-cuota'})
                bets_list.append(bet.text)

            bet = Bet(teams, bets_list)
            ret_bets.append(bet)
        return ret_bets

In [212]:
%%time
reta = Retabet()
reta_tennis = reta.sport_bets("tennis")

ValueError: could not convert string to float: '1,27'

In [8]:
class Bwin(House):
    def __init__(self):
        self.link = "https://sports.bwin.es/es/sports"
        self.sports = {"tennis" : "tenis-5"}

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        while True:
            try:
                driver.find_element_by_css_selector('#main-view > ms-widget-layout > ms-widget-slot.col-9.widget-slot')
                break
            except:
                time.sleep(0.5)
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        table1 = soup.find('ms-widget-slot', attrs={'class':'col-9 widget-slot'})
        table2 = table1.find('ms-tabbed-grid', attrs={'class':'grid-wrapper tabbed-grid-widget'})
        table3 = table2.find('ms-grid')
        table4 = table3.find('ms-event-group', attrs={'class':'event-group'})
        rows = table4.findAll('ms-event', attrs={'class':'grid-event ms-active-highlight'})
        
        ret_bets = []
        for row in rows:
            try:
                row = row.find('div', attrs={'class':'grid-event-wrapper'})
                
                teams = row.find('a', attrs={'class':'grid-info-wrapper fixed'})
                teams = teams.find('ms-event-detail', attrs={'class':'grid-event-detail'})
                teams = teams.find('ms-event-name', attrs={'class':'grid-event-name'})
                teams = teams.find('ms-inline-tooltip')
                teams = teams.find('div', attrs={'class':'participants-pair-game'})
                teams = teams.findAll('div', {'class':['participant-wrapper ng-star-inserted', 'participant-wrapper']})
                teams_final = []
                for team in teams:
                    team = team.find('div', attrs={'class':'participant'})
                    team = team.find(text=True, recursive=False).strip()
                    teams_final.append(team)

                bets = row.find('div', attrs={'class':'grid-group-container'})
                bets = row.find('ms-option-group', attrs={'class':'grid-option-group grid-group'})
                bets = row.findAll('ms-option', attrs={'class':'grid-option'})
                bets_final = []
                for bet, _ in zip(bets, range(2)):
                    bets_final.append(bet.text)

                bet = Bet(teams_final, bets_final)
                ret_bets.append(bet)
            except:
                pass
        return ret_bets

In [209]:
%%time
bwin = Bwin()
bwin_tennis = bwin.sport_bets("tennis")

Wall time: 3.8 s


In [210]:
print(bwin_tennis)

[KAMILLA RAKHIMOVA (1.22) x CLARA BUREL (4.0), CAROLINE GARCIA (1.48) x ASTRA SHARMA (2.55), MARYNA ZANEVSKA (1.83) x STEFANIE VOGELE (1.91), BENOIT PAIRE (1.34) x JUAN PABLO VARILLAS (3.1), JACK SOCK (3.1) x KEVIN ANDERSON (1.34), A. KRAJICEK/V. POSPISIL (1.3) x H. REESE/S. VERBEEK (3.25), ALYCIA PARKS (5.5) x FIONA FERRO (1.14), NIKOLOZ BASILASHVILI (2.45) x LASLO DJERE (1.53), STEFANOS TSITSIPAS (1.14) x FILIP KRAJINOVIC (5.5), VIKTORIA KUZMOVA (3.6) x TEREZA MARTINCOVA (1.28), ZARINA DIYAS (3.25) x CAMILA GIORGI (1.33), YANNICK HANFMANN (1.83) x ARTHUR RINDERKNECH (1.95), FEDERICO CORIA (3.25) x CRISTIAN GARIN (1.33), JENSON BROOKSBY (1.67) x PETER GOJOWCZYK (2.15), JORDAN THOMPSON (1.42) x MAXIME CRESSY (2.85)]


In [202]:
%%time
for bet in reta_tennis:
    cnt, l_f = 0, []
    for player in all_players:
        if player[0] == bet.teams[0][0] and player.split()[-1] == bet.teams[0].split()[-1]:
            cnt += 1
            l_f.append((player, lev.jaro_winkler(bet.teams[0][::-1], player[::-1])))
    print(cnt)
    #l_f = [(p, lev.jaro_winkler(bet.teams[0][::-1], p[::-1])) for p in all_players]
    l_f.sort(key=lambda x: x[1], reverse=True)
    print(bet.teams[0], l_f[:3])

1
F. DELBONIS [('FEDERICO DELBONIS', 0.9832442067736186)]
1
C. TABERNER [('CARLOS TABERNER', 0.9858585858585859)]
1
F. JONES [('FRANCESCA JONES', 0.8866666666666667)]
1
J. NIKLES [('JOHAN NIKLES', 0.9555555555555555)]
1
L. BOCCHI [('LORENZO BOCCHI', 0.946031746031746)]
0
K. PITER / A. RUS []
1
M. HIBI [('MAYO HIBI', 0.9206349206349207)]
0
R. MONTGOMERY (M) []
0
K. SCOTT (M) []
0
A. REAMI []
0
A. BONDAR / D. GALFI []
0
O. LUZ / F. MELIGENI RODRIGUES ALVES []
1
E. YMER [('ELIAS YMER', 0.9095238095238096)]
0
A. BROGAN []
0
L. BEVAN []
1
C. TABUR [('CLÉMENT TABUR', 0.8948717948717949)]
0
A. PANOVA / J. WACHACZYK []
1
T. ETCHEVERRY [('TOMAS MARTIN ETCHEVERRY', 1.0)]
1
A. CAZAUX [('ARTHUR CAZAUX', 0.9504273504273504)]
0
G. BROUWER / J. SELS []
0
M. KESTELBOIM / F. MENA []
1
Y. NISHIOKA / Y. UCHIYAMA [('YASUTAKA UCHIYAMA', 0.9710286576168929)]
1
Y. BONAVENTURE [('YSALINE BONAVENTURE', 1.0)]
1
P. MARTÍN TIFFON [('POL MARTIN TIFFON', 0.9799019607843137)]
1
M. YEROLYMOS [('MARGOT YEROLYMOS', 1.0

In [182]:
lev.jaro_winkler("Novak Djokovic", "Aovok Djokóvíć")

0.692857142857143

In [ ]:
%%time
for bet in bwin_tennis:
    print(bet.teams[0])
    if bet.teams[0] in all_players:
        print("YES")
    else:
        print("NO")

In [169]:
class WilliamHill(House):
    def __init__(self):
        self.link = "https://sports.williamhill.es/betting/es-es"
        self.sports = {"tennis" : "tenis/partidos"}

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.2)
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        matches = soup.findAll('div', attrs={'class':'event'})
        ret_bets = []
        for match in matches:
            teams = match.find('a')
            teams_final, bets_final = [], []
            teams_final = teams['title'].split('₋')
            bets = match.findAll('button', attrs={'class':'btn betbutton oddsbutton'})
            for bet in bets:
                bets_final.append(bet.text)
            ret_bets.append(Bet(teams_final, bets_final))
        return ret_bets

In [177]:
%%time
william_hill = WilliamHill()
william_hill_tennis = william_hill.sport_bets("tennis")

Wall time: 2.57 s


In [178]:
william_hill_tennis

[BENOIT PAIRE (4.5) x JUAN PABLO VARILLAS (1.17),
 ALEXANDER BUBLIK (1.0) x JASON JUNG (34.0),
 AUSTIN KRAJICEK/VASEK POSPISIL (1.36) x HUNTER REESE/SEM VERBEEK (3.0),
 KAMILLA RAKHIMOVA (1.17) x CLARA BUREL (4.5),
 CAROLINE GARCIA (1.4) x ASTRA SHARMA (2.75),
 ALYCIA PARKS (6.0) x FIONA FERRO (1.12),
 ZARINA DIYAS (3.3) x CAMILA GIORGI (1.33),
 MANDY MINELLA/STEFANIE VOEGELE (4.75) x ERI HOZUMI/SHUAI ZHANG (1.15),
 ALEXANDRA PANOVA/JULIA WACHACZYK (1.5) x FRANCESCA DI LORENZO/ASTRA SHARMA (2.5),
 TALLON GRIEKSPOOR (1.01) x MICHAEL VRBENSKY (21.0),
 JOHAN NIKLES (1.91) x LUKAS ROSOL (1.8),
 ALEXANDER RITSCHARD (1.02) x GIOVANNI FONIO (13.0),
 ALESSANDRO MOTTI/JULIAN OCLEPPO (2.1) x ALESSIO DE BERNARDIS/FRANCESCO PASSARO (1.67),
 MARCO BORTOLOTTI/CRISTIAN RODRÍGUEZ (1.8) x FRANCESCO FORTI/GIULIO ZEPPIERI (1.91),
 FACUNDO DIAZ ACOSTA/ALEXANDER MERINO (2.62) x TOMÁS MARTÍN ETCHEVERRY/THIAGO AGUSTIN TIRANTE (1.44),
 EKATERINA SHALIMOVA (3.25) x ANASTASIA TIKHONOVA (1.33),
 MARIAM BOLKVADZE

In [11]:
driver.get("https://m.apuestas.codere.es/deportes/index.htm#/HomePage")

In [13]:
driver.get("https://betway.es/es/sports")

In [14]:
<span class="teamNameFirstPart teamNameHomeTextFirstPart">Elias Ymer</span>

SyntaxError: invalid syntax (<ipython-input-14-1cfd38ecb6f9>, line 1)